In [1]:
import numpy as np
from scipy.special import erf
import os
import sys
import time
import copy
import logging
import matplotlib.pyplot as plt

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from utils.constants import WORKERS, BATCH_SAMPLES
from utils.commons import convert_bytes_to_mb, convert_bytes_to_gb
from Dataset.flight_dataset_for_LS import load_flight_dataset_matrices_for_LS, save_flight_dataset_matrices_for_LS
from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from Dataset.songs_dataset_for_LS import load_song_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b, twoNorm, data_normalize_by_sample

from RP_mechanisms.optim_RP import OptimalRP_mech, parallel_gen_samples
from RP_mechanisms.alt19_RP import ALT19RP_mech
from RP_mechanisms.RP_accuracy_estimator import RP_accuracy_estimator
from RP_mechanisms.isit18_RP import ISIT18RP_mech

from analysis.commons import compute_xopt, split_to_B_b

In [3]:
logfile_path = os.path.join(log_dir, 'RP-flight-default.log')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path),
        logging.StreamHandler()
    ]
)

def store_array_str(data_array):
    converted_str = np.array_str(data_array)
    converted_str = converted_str.replace(' ',',')
    return converted_str

file_X_path = os.path.join(project_dir, "Dataset", "flight-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "flight-LR-y.txt")

X = None
y = None
try:
    X, y = load_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    save_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_flight_dataset_matrices_for_LS()
    
B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [4]:
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : 1200,
        'CI' : 0.95,
        'bootstrap_samples': 100
    }
    
    return kwargs

kwargs = generate_default_configuration()

In [5]:
mech = OptimalRP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100
sigma = mech.find_minimal_sigma(epsilon, delta)
# samples = mech._gen_samples(epsilon, delta=delta, num_samples=num_samples)
print(sigma)

0.7452001524823914


/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in double_scalars
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)


In [6]:
isit_mech = ISIT18RP_mech(kwargs)

epsilon = 1
delta = 1/isit_mech.n
num_samples = 100
tic = time.perf_counter()
isit_samples = parallel_gen_samples(isit_mech, epsilon, delta, num_samples, workers=50)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:12:54,786 cost 38.7758 seconds


In [7]:
tic = time.perf_counter()
accuracy_estimator = RP_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_pairwise_distance_acc(isit_samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:12:54,890 bootstrap_result(lb_one_sided=110.68420894063259, lb_two_sided=110.58410772431421, ub_one_sided=111.28590373196879, ub_two_sided=111.35650601559831)
2024-12-09 14:12:54,891 cost 0.0961 seconds


In [8]:
mech = OptimalRP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = parallel_gen_samples(mech, epsilon, delta, num_samples, workers=50)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:13:45,739 cost 47.1970 seconds


In [9]:
tic = time.perf_counter()
accuracy_estimator = RP_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_pairwise_distance_acc(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:13:45,836 bootstrap_result(lb_one_sided=0.996017261037031, lb_two_sided=0.9958141311578566, ub_one_sided=1.0015874677844387, ub_two_sided=1.001790762886305)
2024-12-09 14:13:45,837 cost 0.0914 seconds


In [10]:
alt_mech = ALT19RP_mech(kwargs)
# samples = alt_mech._gen_samples(epsilon, delta=delta, num_samples=num_samples)

tic = time.perf_counter()
alt_samples = parallel_gen_samples(alt_mech, epsilon, delta, num_samples, workers=50)
toc = time.perf_counter()

logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:14:26,233 cost 40.3902 seconds


In [11]:
tic = time.perf_counter()
accuracy_estimator = RP_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_pairwise_distance_acc(alt_samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:14:26,327 bootstrap_result(lb_one_sided=3.997872156884723, lb_two_sided=3.996592487297705, ub_one_sided=4.016087036559668, ub_two_sided=4.016722730686742)
2024-12-09 14:14:26,328 cost 0.0857 seconds


In [12]:
tic = time.perf_counter()
samples = parallel_gen_samples(mech, epsilon, delta=1, num_samples=num_samples, workers=50)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:15:05,024 cost 38.6906 seconds


In [13]:
tic = time.perf_counter()
accuracy_estimator = RP_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_pairwise_distance_acc(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2024-12-09 14:15:05,117 bootstrap_result(lb_one_sided=0.9978168817085027, lb_two_sided=0.9974493033298009, ub_one_sided=1.00314995157654, ub_two_sided=1.0032662842395084)
2024-12-09 14:15:05,118 cost 0.0878 seconds
